
Alembic の初期設定を行い、最初のテーブルを作成するまでの手順

1. 各種ファイルを作成
    ```console
    $ alembic init migrations
    ```
2. `models.py` を作成する
3. `migrations/env.py` を設定する
    - `target_metadata` の値に `models.py` で作成したベースモデルを指定する
    - DB 接続情報を環境変数から取得できるようにする
4. マイグレーションファイルを生成する
    ```console
    $ alembic revision --autogenerate -m "Create tables" --rev-id $(date -u +"%Y%m%d_%H%M%S")
    ```
5. マイグレーションファイルを適用する
    ```console
    $ alembic upgrade head
    ```